Import libraries

In [39]:
import numpy as np
from numba import cuda, float32
import math
import time


Set block and matrix size

In [40]:
# Block and matrix size
TPB = 16
N = 256


Create matrices and transfer to GPU

In [41]:
# Matrix creation
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)

# Transfer matrices to the GPU
A_device = cuda.to_device(A)
B_device = cuda.to_device(B)
C_device = cuda.device_array((N, N), dtype=np.float32)  # Resultant matrix on the GPU


In [42]:
import numba
from numba import cuda
print("CUDA is available:", cuda.is_available())


CUDA is available: True


Define the CUDA kernel with shared memory

In [43]:
# Define the CUDA kernel with shared memory
@cuda.jit
def matmul_shared_kernel(A, B, C):
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y
    if x >= C.shape[0] or y >= C.shape[1]:
        return

    temp = 0.0
    for i in range(int(A.shape[1] / TPB)):
        sA[ty, tx] = A[x, ty + i * TPB]
        sB[ty, tx] = B[tx + i * TPB, y]
        cuda.syncthreads()
        for k in range(TPB):
            temp += sA[ty, k] * sB[k, tx]
        cuda.syncthreads()
    C[x, y] = temp


Configure blocks and grids, and measure execution time

In [44]:
# Configure blocks and grids
blockspergrid = (math.ceil(N / TPB), math.ceil(N / TPB))

# Measure execution time with cuda.event
start = cuda.event()
end = cuda.event()
start.record()

# Execute the kernel
matmul_shared_kernel[blockspergrid, (TPB, TPB)](A_device, B_device, C_device)

end.record()
end.synchronize()

# Calculate elapsed time
time_elapsed = cuda.event_elapsed_time(start, end)
print(f"Kernel execution time: {time_elapsed} ms")


Kernel execution time: 297.3147277832031 ms


Retrieve and display data from GPU

In [45]:
# Retrieve data from GPU for verification
C = C_device.copy_to_host()
print("Matrix multiplication result:\n", C)


Matrix multiplication result:
 [[65.03665  65.359726 62.63055  ... 69.617645 63.310112 67.872375]
 [68.68004  67.239265 61.43986  ... 65.68339  61.150177 63.23375 ]
 [64.672676 66.36231  62.12741  ... 69.57043  63.143124 67.746185]
 ...
 [66.34234  63.369644 67.45249  ... 66.56866  69.88551  59.975952]
 [63.4768   61.443436 65.98586  ... 66.416016 70.80537  63.430073]
 [62.882126 61.226448 65.27813  ... 66.51149  72.234314 60.80983 ]]


Prompt the user for matrix size and shared memory usage

In [46]:
# Prompt the user for matrix size and whether shared memory should be used
N = int(input("Enter matrix size (N x N): "))
use_shared = input("Use shared memory? (yes/no): ").lower() == 'yes'
TPB = 16  # Fixed block size

# Matrix creation
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)


Enter matrix size (N x N): 5000
Use shared memory? (yes/no): yes


Kernel without shared memory

In [47]:
# Kernel without shared memory
@cuda.jit
def matmul_kernel(A, B, C):
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        sum = 0.0
        for k in range(A.shape[1]):
            sum += A[row, k] * B[k, col]
        C[row, col] = sum


Kernel with shared memory

In [48]:
# Kernel with shared memory
@cuda.jit
def matmul_shared_kernel(A, B, C):
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    x, y = cuda.grid(2)
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y
    if x >= C.shape[0] or y >= C.shape[1]:
        return
    temp = 0.0
    for i in range(int(A.shape[1] / TPB)):
        sA[ty, tx] = A[x, ty + i * TPB]
        sB[ty, tx] = B[tx + i * TPB, y]
        cuda.syncthreads()
        for k in range(TPB):
            temp += sA[ty, k] * sB[k, tx]
        cuda.syncthreads()
    C[x, y] = temp


Matrix multiplication execution function

In [49]:
# Perform matrix multiplication
def matmul_gpu(A, B, use_shared):
    A_device = cuda.to_device(A)
    B_device = cuda.to_device(B)
    C_device = cuda.device_array((N, N), dtype=np.float32)
    blockspergrid = (math.ceil(N / TPB), math.ceil(N / TPB))
    start = time.time()
    if use_shared:
        matmul_shared_kernel[blockspergrid, (TPB, TPB)](A_device, B_device, C_device)
    else:
        matmul_kernel[blockspergrid, (TPB, TPB)](A_device, B_device, C_device)
    cuda.synchronize()
    end = time.time()
    print(f"Execution time: {end - start:.4f} seconds")
    return C_device.copy_to_host()


Calculate and display result

In [50]:
# Calculate and display result
C = matmul_gpu(A, B, use_shared)
print("Matrix multiplication result:\n", C)


Execution time: 2.5950 seconds
Matrix multiplication result:
 [[1253.4149 1264.2921 1255.6302 ... 1282.8734 1275.8226 1279.1812]
 [1234.7328 1253.2761 1233.0634 ... 1264.9348 1271.144  1265.0441]
 [1241.063  1238.1512 1236.2037 ... 1151.7234 1149.4824 1139.849 ]
 ...
 [1253.654  1367.8208 1462.7887 ... 1379.2767 1380.4052 1182.7166]
 [1266.3993 1369.3339 1474.8586 ... 1403.8734 1468.8204 1175.8052]
 [1242.7612 1355.1373 1442.8925 ... 1459.5002 1616.5847 1283.6373]]


Define matrix and tile size

In [51]:
# Define matrix and tile size
N = 10000
TILE_DIM = 32


Tiled kernel for large matrix multiplication

In [52]:
@cuda.jit
def matmul_tiled_kernel(A, B, C):
    # Declare shared memory for blocks
    tile_A = cuda.shared.array((TILE_DIM, TILE_DIM), dtype=float32)
    tile_B = cuda.shared.array((TILE_DIM, TILE_DIM), dtype=float32)

    # Thread, block, and tile indices
    bx, by = cuda.blockIdx.x, cuda.blockIdx.y
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y
    x, y = tx + bx * TILE_DIM, ty + by * TILE_DIM

    sum = 0.0
    for i in range((N + TILE_DIM - 1) // TILE_DIM):
        # Load tiles into shared memory
        if i * TILE_DIM + tx < N and y < N:
            tile_A[ty, tx] = A[y, i * TILE_DIM + tx]
        else:
            tile_A[ty, tx] = 0

        if i * TILE_DIM + ty < N and x < N:
            tile_B[ty, tx] = B[i * TILE_DIM + ty, x]
        else:
            tile_B[ty, tx] = 0

        cuda.syncthreads()  # Synchronize threads in block

        # Multiply tiles
        for k in range(TILE_DIM):
            sum += tile_A[ty, k] * tile_B[k, tx]

        cuda.syncthreads()  # Synchronize before loading the next tile

    if x < N and y < N:
        C[y, x] = sum


Matrix creation and transfer to GPU for tiled kernel

In [53]:
# Matrix creation
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)
C = np.zeros((N, N), dtype=np.float32)

# Transfer matrices to GPU
A_device = cuda.to_device(A)
B_device = cuda.to_device(B)
C_device = cuda.to_device(C)


Configure thread and grid blocks, launch CUDA kernel, and retrieve results

In [54]:
# Configure thread and grid blocks
threads_per_block = (TILE_DIM, TILE_DIM)
blocks_per_grid = (math.ceil(N / TILE_DIM), math.ceil(N / TILE_DIM))

# Launch CUDA kernel
matmul_tiled_kernel[blocks_per_grid, threads_per_block](A_device, B_device, C_device)

# Retrieve results from GPU
C = C_device.copy_to_host()


Display sample result

In [55]:
# Display results
print("Matrix multiplication result (sample):\n", C[:5, :5])


Matrix multiplication result (sample):
 [[2487.4058 2469.4377 2496.5815 2499.6643 2496.6184]
 [2517.4097 2467.949  2525.5752 2510.6067 2512.0613]
 [2494.2    2470.1106 2518.3276 2520.9238 2502.7434]
 [2497.048  2479.4536 2522.7744 2515.4663 2503.5168]
 [2480.5332 2437.5642 2499.915  2499.4597 2492.8352]]


Validation function

In [56]:
def validate_results(A, B, C):
    # Compute matrix product with NumPy for validation
    C_numpy = np.dot(A, B)
    if np.allclose(C, C_numpy, atol=1e-5):
        print("Validation successful: GPU and NumPy results are similar.")
    else:
        print("Validation error: Results differ.")

# Use this function after retrieving C from the GPU
validate_results(A, B, C)


Validation successful: GPU and NumPy results are similar.
